<a href="https://colab.research.google.com/github/sanketh96/Chatbot-2.0/blob/master/NER_LSTM_CRF(kaggle_augmented).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-q6gxgw6u
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0g410kx/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import copy

In [0]:
data = pd.read_csv('gdrive/My Drive/augmented_new.csv', encoding="latin1")

In [0]:
data = data.fillna(method="ffill")


In [0]:
data.tail(10)


,Unnamed: 0,POS,final_tag,sentence_id,word
627320,406397,NNS,o,25596,clashes
627321,406398,IN,o,25596,with
627322,406399,NN,o,25596,security
627323,406400,NNS,o,25596,forces
627324,406401,IN,o,25596,during
627325,406402,NN,o,25596,post-election
627326,406403,",",o,25596,","
627327,406404,JJ,o,25596,anti-government
627328,406405,NNS,o,25596,demonstrations
627329,406406,.,o,25596,.


In [0]:
words = list(set(data["word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

32709

In [0]:
tags = list(set(data["final_tag"].values))
n_tags = len(tags); n_tags

17

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["final_tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(data)


In [0]:
sent = getter.get_next()


In [0]:
print(sent)


[('30', 'CD', 'o'), ('P.M', 'NNP', 'o'), ('National', 'NNP', 'o'), ('Savings', 'NNP', 'o'), ('Certificates', 'NNS', 'o'), ('(', '(', 'o'), ('NSC', 'NNP', 'o'), (')', 'SYM', 'o')]


In [0]:
sentences = getter.sentences


In [0]:
cleaned_sentences = copy.deepcopy(sentences)
for outer_index in range(len(sentences)):
  for inner_index in range(len(sentences[outer_index])):
    if type(sentences[outer_index][inner_index][0]) is not str:
      print(sentences[outer_index])
      print(sentences[outer_index][inner_index])
      temp_list = list(sentences[outer_index][inner_index])
      temp_list[0] = ' '
      cleaned_sentences[outer_index][inner_index] = tuple(temp_list)
      print("After cleaning : ")
      print(cleaned_sentences[outer_index])
      print(cleaned_sentences[outer_index][inner_index])

[('39', 'CD', 'o'), ('in', 'IN', 'o'), (')', 'SYM', 'o'), ('per', 'IN', 'o'), ('hour', 'NN', 'o'), ('[', '(', 'o'), ('95', 'CD', 'o'), (']', ')', 'o'), ('[', '(', 'o'), ('96', 'CD', 'o'), (']', ')', 'o'), ('*', 'SYM', 'o'), ('Heavy', 'NNP', 'o'), ('rain', 'NN', 'o'), ('Ã¢Â\x80Â\x94', 'NN', 'o'), ('when', 'WRB', 'o'), ('the', 'DT', 'o'), ('precipitation', 'NN', 'o'), ('rate', 'NN', 'o'), ('is', 'VBZ', 'o'), (nan, 'SYM', 'o'), ('7', 'CD', 'o'), ('.', '.', 'o')]
(nan, 'SYM', 'o')
After cleaning : 
[('39', 'CD', 'o'), ('in', 'IN', 'o'), (')', 'SYM', 'o'), ('per', 'IN', 'o'), ('hour', 'NN', 'o'), ('[', '(', 'o'), ('95', 'CD', 'o'), (']', ')', 'o'), ('[', '(', 'o'), ('96', 'CD', 'o'), (']', ')', 'o'), ('*', 'SYM', 'o'), ('Heavy', 'NNP', 'o'), ('rain', 'NN', 'o'), ('Ã¢Â\x80Â\x94', 'NN', 'o'), ('when', 'WRB', 'o'), ('the', 'DT', 'o'), ('precipitation', 'NN', 'o'), ('rate', 'NN', 'o'), ('is', 'VBZ', 'o'), (' ', 'SYM', 'o'), ('7', 'CD', 'o'), ('.', '.', 'o')]
(' ', 'SYM', 'o')
[('0', 'CD', 'o'),

In [0]:
cleaned_words = []
for sent in cleaned_sentences:
  for word in sent:
    cleaned_words.append(word[0])
words = list(set(cleaned_words))

In [0]:
words.sort()

In [0]:
words

[' ',
 '!',
 '"',
 '#',
 '#NAME?',
 '$',
 '$1.25',
 '$1.3',
 '$1.6',
 '$1.7',
 '$10',
 '$131,000',
 '$192,000',
 '$2.1',
 '$2.4',
 '$31.5',
 '$4.2',
 '$42.1',
 '$5.5',
 '$50,000',
 '$57.5',
 '$700,000',
 '$75.09',
 '$8',
 '$900,000',
 '%',
 '&',
 "'",
 "'70s",
 "'ll",
 "'m",
 "'s",
 '(',
 ')',
 ")'s",
 '*',
 '+',
 ',',
 ",'summa",
 '-',
 '--',
 '--an',
 '-.',
 '-0',
 '-05-27',
 '-11',
 '-9',
 '-LRB-',
 '-LSB-',
 '-Phase',
 '-RRB-',
 '-RSB-',
 '-for',
 '-in',
 '.',
 '...',
 '..next',
 '/',
 '0',
 '0-3',
 '0-5',
 '0-64',
 '0.01',
 '0.02',
 '0.04',
 '0.060092593',
 '0.068171296',
 '0.068217593',
 '0.068275463',
 '0.08',
 '0.1',
 '0.110474537',
 '0.12',
 '0.19',
 '0.2',
 '0.3',
 '0.35',
 '0.5',
 '0.6',
 '0.7',
 '0.8',
 '0.82',
 '00',
 '0000',
 '0039',
 '00684-11-26November',
 '00887-08-22August',
 '0098',
 '01-Feb',
 '01-Jan',
 '01-Jun',
 '01096-12-11December',
 '01361-07-26July',
 '01498-09-11September',
 '01605-02-03February',
 '01707-10-28October',
 '01854-12-23December',
 '02',
 '02-Fe

In [0]:
tags.sort()

In [0]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}


In [0]:
word2idx["Obama"]


11202

In [0]:
tag2idx["B-LOC"]


4

In [0]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in cleaned_sentences]

In [0]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)


In [0]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]


In [0]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["o"])


In [0]:
from keras.utils import to_categorical


In [0]:
y = [to_categorical(i, num_classes=n_tags) for i in y]


In [0]:
from sklearn.model_selection import train_test_split


In [0]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)


In [0]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [0]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(input, out)


In [0]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])


/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [0]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 75, 20)            654200    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 75, 100)           28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 75, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 75, 17)            1190      
Total params: 688,840
Trainable params: 688,840
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5,
                    validation_split=0.1, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 20595 samples, validate on 2289 samples
Epoch 1/5
20595/20595 [==============================] - 116s 6ms/step - loss: 0.2220 - crf_viterbi_accuracy: 0.9421 - val_loss: 0.1248 - val_crf_viterbi_accuracy: 0.9627
Epoch 2/5
20595/20595 [==============================] - 112s 5ms/step - loss: 0.0983 - crf_viterbi_accuracy: 0.9690 - val_loss: 0.0781 - val_crf_viterbi_accuracy: 0.9743
Epoch 3/5
20595/20595 [==============================] - 141s 7ms/step - loss: 0.0664 - crf_viterbi_accuracy: 0.9770 - val_loss: 0.0632 - val_crf_viterbi_accuracy: 0.9771
Epoch 4/5
20595/20595 [==============================] - 139s 7ms/step - loss: 0.0533 - crf_viterbi_accuracy: 0.9802 - val_loss: 0.0567 - val_crf_viterbi_accuracy: 0.9782
Epoch 5/5
20595/20595 [==============================] - 221s 11ms/step - loss: 0.0459 - crf_viterbi_accuracy: 0.9819 - val_loss: 0.0509 - val_crf_viterbi_accuracy: 0.9800


In [0]:
!pip install seqeval

In [0]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [0]:
test_pred = model.predict(X_te, verbose=1)


2543/2543 [==============================] - 10s 4ms/step


In [0]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [0]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))


F1-score: 75.9%


In [0]:
print(classification_report(test_labels, pred_labels))


             precision    recall  f1-score   support

        LOC       0.78      0.80      0.79      1940
        PER       0.74      0.70      0.72      1590
         DT       0.86      0.75      0.80      1226
          o       0.81      0.76      0.78      8703
        GPE       0.90      0.78      0.84      1119
        ORG       0.65      0.52      0.58      1117
        ART       0.00      0.00      0.00        59
        NUM       0.51      0.17      0.25       302
        EVE       0.11      0.06      0.08        48

avg / total       0.79      0.73      0.75     16104



In [0]:
i = 1901
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_te[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_te[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], tags[t], tags[pred]))

Word           ||True ||Pred
Indian         : B-GPE B-GPE
officials      : o     o
say            : o     o
the            : o     o
long-running   : o     o
Maoist         : B-ORG o
insurgency     : o     o
,              : o     o
which          : o     o
has            : o     o
spread         : o     o
to             : o     o
20             : o     o
of             : o     o
the            : o     o
country        : o     o
's             : o     o
28             : o     o
states         : o     o
,              : o     o
is             : o     o
India          : B-LOC B-GPE
's             : o     o
most           : o     o
serious        : o     o
domestic       : o     o
security       : o     o
issue          : o     o
.              : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬   : o     o
Ã­ÂÂÃ¬ÂÂ¬ 

In [0]:
test_sentence = ["Hawking", "was", "a", "Fellow", "of", "the", "Royal", "Society", ",", "a", "lifetime", "member",
                 "of", "the", "Pontifical", "Academy", "of", "Sciences", ",", "and", "a", "recipient", "of",
                 "the", "Presidential", "Medal", "of", "Freedom", ",", "the", "highest", "civilian", "award",
                 "in", "the", "United", "States", "."]

In [0]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

In [0]:
p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction
Hawking        : B-ART
was            : o    
a              : o    
Fellow         : o    
of             : o    
the            : o    
Royal          : B-ORG
Society        : I-ORG
,              : o    
a              : o    
lifetime       : o    
member         : o    
of             : o    
the            : o    
Pontifical     : o    
Academy        : o    
of             : o    
Sciences       : o    
,              : o    
and            : o    
a              : o    
recipient      : o    
of             : o    
the            : o    
Presidential   : o    
Medal          : o    
of             : o    
Freedom        : o    
,              : o    
the            : o    
highest        : o    
civilian       : o    
award          : o    
in             : o    
the            : o    
United         : B-LOC
States         : I-LOC
.              : o    
